In [ ]:
# Importar bibliotecas
import streamlit as st
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from snowflake.snowpark.context import get_active_session

session = get_active_session()

In [ ]:
# Buscar dados do Snowflake
query = """
SELECT 
    DATA_NEGOCIACAO,
    PRECO_ABERTURA,
    PRECO_MAXIMO,
    PRECO_MINIMO,
    PRECO_FECHAMENTO,
    VOLUME
FROM BBAS3.PUBLIC.FATO_ACOES_REAL
WHERE DATA_NEGOCIACAO >= DATEADD(month, -12, CURRENT_DATE())
ORDER BY DATA_NEGOCIACAO
"""

df = session.sql(query).to_pandas()
st.write(f"✅ {len(df)} registros carregados")

In [ ]:
# Calcular Bollinger Bands
df['SMA20'] = df['PRECO_FECHAMENTO'].rolling(window=20).mean()
df['STD20'] = df['PRECO_FECHAMENTO'].rolling(window=20).std()
df['BB_UPPER'] = df['SMA20'] + (df['STD20'] * 2)
df['BB_LOWER'] = df['SMA20'] - (df['STD20'] * 2)

st.write("✅ Bollinger Bands calculados")

In [ ]:
# Calcular RSI (14 períodos)
delta = df['PRECO_FECHAMENTO'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))

st.write("✅ RSI calculado")

In [ ]:
# Calcular MACD (12, 26, 9)
df['EMA12'] = df['PRECO_FECHAMENTO'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['PRECO_FECHAMENTO'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA12'] - df['EMA26']
df['SIGNAL'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['HISTOGRAM'] = df['MACD'] - df['SIGNAL']

st.write("✅ MACD calculado")

In [ ]:
# Criar dashboard com 4 painéis
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.4, 0.2, 0.2, 0.2],
    subplot_titles=(
        'Preço + Bollinger Bands',
        'RSI (14)',
        'MACD',
        'Volume'
    )
)

# PAINEL 1: Preço + Bollinger Bands
fig.add_trace(
    go.Scatter(
        x=df['DATA_NEGOCIACAO'],
        y=df['PRECO_FECHAMENTO'],
        name='Preço',
        line=dict(color='white', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['DATA_NEGOCIACAO'],
        y=df['BB_UPPER'],
        name='BB Superior',
        line=dict(color='gray', width=1, dash='dash'),
        showlegend=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['DATA_NEGOCIACAO'],
        y=df['BB_LOWER'],
        name='BB Inferior',
        line=dict(color='gray', width=1, dash='dash'),
        fill='tonexty',
        fillcolor='rgba(128,128,128,0.2)',
        showlegend=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['DATA_NEGOCIACAO'],
        y=df['SMA20'],
        name='SMA20',
        line=dict(color='orange', width=1.5)
    ),
    row=1, col=1
)

# PAINEL 2: RSI
fig.add_trace(
    go.Scatter(
        x=df['DATA_NEGOCIACAO'],
        y=df['RSI'],
        name='RSI',
        line=dict(color='purple', width=2)
    ),
    row=2, col=1
)

fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1, opacity=0.5)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1, opacity=0.5)
fig.add_hrect(y0=70, y1=100, line_width=0, fillcolor="red", opacity=0.1, row=2, col=1)
fig.add_hrect(y0=0, y1=30, line_width=0, fillcolor="green", opacity=0.1, row=2, col=1)

# PAINEL 3: MACD
fig.add_trace(
    go.Scatter(
        x=df['DATA_NEGOCIACAO'],
        y=df['MACD'],
        name='MACD',
        line=dict(color='blue', width=2)
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df['DATA_NEGOCIACAO'],
        y=df['SIGNAL'],
        name='Signal',
        line=dict(color='orange', width=1.5)
    ),
    row=3, col=1
)

colors = ['green' if h > 0 else 'red' for h in df['HISTOGRAM']]
fig.add_trace(
    go.Bar(
        x=df['DATA_NEGOCIACAO'],
        y=df['HISTOGRAM'],
        name='Histogram',
        marker_color=colors,
        opacity=0.5
    ),
    row=3, col=1
)

# PAINEL 4: Volume
volume_colors = ['green' if df['PRECO_FECHAMENTO'].iloc[i] >= df['PRECO_ABERTURA'].iloc[i] 
                 else 'red' for i in range(len(df))]

fig.add_trace(
    go.Bar(
        x=df['DATA_NEGOCIACAO'],
        y=df['VOLUME'],
        name='Volume',
        marker_color=volume_colors,
        showlegend=False
    ),
    row=4, col=1
)

# Configurar layout
fig.update_layout(
    title='BBAS3 - Dashboard Completo de Indicadores Técnicos',
    template='plotly_dark',
    height=1000,
    showlegend=True,
    hovermode='x unified',
    xaxis4_title='Data'
)

fig.update_yaxes(title_text="Preço (R$)", row=1, col=1)
fig.update_yaxes(title_text="RSI", row=2, col=1, range=[0, 100])
fig.update_yaxes(title_text="MACD", row=3, col=1)
fig.update_yaxes(title_text="Volume", row=4, col=1)

st.plotly_chart(fig, use_container_width=True)

In [ ]:
# Sinais de Trading
st.subheader("🎯 Sinais de Trading (Último Pregão)")

ultimo = df.iloc[-1]
penultimo = df.iloc[-2]

col1, col2, col3, col4 = st.columns(4)

with col1:
    if ultimo['RSI'] > 70:
        st.error(f"RSI: {ultimo['RSI']:.1f} - SOBRECOMPRADO ⚠️")
    elif ultimo['RSI'] < 30:
        st.success(f"RSI: {ultimo['RSI']:.1f} - SOBREVENDIDO 💰")
    else:
        st.info(f"RSI: {ultimo['RSI']:.1f} - NEUTRO")

with col2:
    if ultimo['MACD'] > ultimo['SIGNAL'] and penultimo['MACD'] <= penultimo['SIGNAL']:
        st.success("MACD: CRUZAMENTO DE ALTA 🚀")
    elif ultimo['MACD'] < ultimo['SIGNAL'] and penultimo['MACD'] >= penultimo['SIGNAL']:
        st.error("MACD: CRUZAMENTO DE BAIXA 📉")
    else:
        st.info("MACD: SEM CRUZAMENTO")

with col3:
    if ultimo['PRECO_FECHAMENTO'] > ultimo['BB_UPPER']:
        st.warning("BB: ACIMA BANDA SUPERIOR")
    elif ultimo['PRECO_FECHAMENTO'] < ultimo['BB_LOWER']:
        st.success("BB: ABAIXO BANDA INFERIOR")
    else:
        st.info("BB: DENTRO DAS BANDAS")

with col4:
    distancia_sma = ((ultimo['PRECO_FECHAMENTO'] / ultimo['SMA20']) - 1) * 100
    if distancia_sma > 5:
        st.warning(f"Preço {distancia_sma:.1f}% acima SMA20")
    elif distancia_sma < -5:
        st.success(f"Preço {abs(distancia_sma):.1f}% abaixo SMA20")
    else:
        st.info(f"Preço {distancia_sma:+.1f}% vs SMA20")